In [ ]:
"""This example makes a reactor geometry and a neutronics model, the addition
of a Python for loop allow a parameter sweep of the neutronics results for
different geometries. The distance between the plasma and center column is
varried while simulating the impact on the heat depositied in the center
column."""

import matplotlib.pyplot as plt
import openmc
import openmc_dagmc_wrapper as odw


source = openmc.Source()
# sets the location of the source to x=0 y=0 z=0
source.space = openmc.stats.Point((my_reactor.major_radius, 0, 0))
# sets the direction to isotropic
source.angle = openmc.stats.Isotropic()
# sets the energy distribution to 100% 14MeV neutrons
source.energy = openmc.stats.Discrete([14e6], [1])

# makes the neutronics model and assigns basic materials to each
# component
neutronics_model = odw.NeutronicsModel(
    h5m_filename=my_reactor.export_h5m(),
    source=source,
    materials={
        'DT_plasma': 'DT_plasma',
        'inboard_tf_coils_mat': 'eurofer',
        'center_column_shield_mat': 'eurofer',
        'divertor_mat': 'eurofer',
        'firstwall_mat': 'eurofer',
        'blanket_mat': 'Li4SiO4'},
    cell_tallies=['heating'],
)

# starts the neutronics simulation
h5m_filename = neutronics_model.simulate(
    simulation_batches=2,
    simulation_particles_per_batch=10,  # this would need increasing to get a accurate result
)

# extracts and processes the results from the statepoint file
results = neutronics_model.process_results(statepoint_filename=h5m_filename)

# converts the results to mega watts
total_heat_in_MW = results['firstwall_mat_heating']['Watts']['result'] / 1e6

